In [ ]:
import snap
import random
import zlib
import csv
from scipy import stats
from scipy.optimize import curve_fit
import statistics
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import numpy as np
from timeit import default_timer as timer
from pathlib import Path
import time

In [ ]:
method = 1
def get_color(vertex, k):
    if method == 0:
        return vertex % k 
    if method == 1:
        return perm[vertex % 999] % k 
    elif method == 2:
        return zlib.adler32(str(vertex + seed).encode('UTF-8')) % k

def graph_sampling(stream, k):
    global seed
    seed = random.randint(0,1000000)
    global perm
    perm = list(range(999))
    random.shuffle(perm)
    G = snap.TUNGraph.New()
    for edge in stream:
        u = int(edge[0])
        v = int(edge[1])
        if (get_color(u, k) == get_color(v, k)):
            if (G.IsNode(u) == False):
                G.AddNode(u)
            if (G.IsNode(v) == False):
                G.AddNode(v)
            G.AddEdge(u, v)
    return G

def graph_sampling_file(path, seperator, k):
    f = open(path, newline='')
    stream = csv.reader(f, delimiter=seperator)
    
    return graph_sampling(stream, k)

In [ ]:
# returns an array of tuples (node, percentage_of_triangles_it_participates_in) and the total number of triangles
def calc_triangle_participation(G,k=1):
    multiplier = k**2
    
    counts = []
    total = 0
    TriadV = snap.TIntTrV()
    # Compute triangles and wedges for every node and store the information in TriadV
    snap.GetTriads(G, TriadV)
    # Convert the information to an array of tuples
    for t in TriadV:
        counts.append([t.GetVal1(), t.GetVal2()*multiplier])
        total = total + t.GetVal2()
    return counts, total/3

In [ ]:
def create_dist(nodes):
    frequency = np.unique(nodes, return_counts=True)
    return dict(zip(frequency[0], frequency[1]))

In [ ]:
def ks_test(approx_participation, exact_triangle_participation): 
    approx_count = []
    for triple in approx_participation:
        approx_count.append(triple[1])
        
    # extract exact number of triangles
    triangle_count = []
    for node in exact_triangle_participation:
        triangle_count.append(node[1])
    
    stat = stats.ks_2samp(approx_count, triangle_count)
    
    return stat.statistic

In [ ]:
def takeSecond(elem):
    return elem[1]

def cut_threshold(triangle_participation, threshold):
    filtered = []
    for node in triangle_participation:
        if node[1] >= threshold:
            filtered.append(node)
    filtered.sort(key=takeSecond)
    return filtered

# Experiments

In [ ]:
# get something similar like the graphic there: https://en.wikipedia.org/wiki/Kolmogorov%E2%80%93Smirnov_test
def cumulative_probability(dist, n):
    distc = {}
    previous = 0
    for key, value in dist.items():
        cumulated = (previous + value)
        distc[key] = cumulated / n
        previous = cumulated
    return distc

def run_experiment(path, k_arr, out_path=False, show_cmf=False, show_dist=False):
    start = timer()
    exact_participation, total_exact = calc_triangle_participation(snap.LoadEdgeList(snap.PUNGraph, f'data/{path}', 0, 1, '\t'))
    end = timer()
    time_exact = (end - start)
    
    # key is k, value is time
    times_apx_sampling = {}
    times_apx_counting = {}
    times_apx_total = {}
    times_cmf = {} # TODO: seperate exact and approx
    
    # key is k, value is statistic
    ks_statistics = {}
    
    distribution = create_dist(exact_participation)
    distribution_c = cumulative_probability(distribution, len(exact_participation))
    
    approx_dist_c_k = {}
    
    for k in k_arr:        
        start = timer()
        Gs = graph_sampling_file(f'data/{path}', '\t', k)
        end = timer()
        times_apx_sampling[k] = (end - start)
        
        start = timer()
        t_p, total = calc_triangle_participation(Gs, k)
        end = timer()
        times_apx_counting[k] = (end - start)
        
        
        start = timer()
        approx_dist = create_dist(t_p)
        approx_dist_c_k[k] = cumulative_probability(approx_dist, len(t_p))
        end = timer()
        times_cmf[k] = (end - start)
        
        ks_statistics[k] = ks_test(t_p, exact_participation)
        
        if show_dist:
            plt.scatter(list(distribution.keys()), distribution.values(), color="blue", s=10, label='Exact distribution')
            plt.scatter(list(approx_dist.keys()), approx_dist.values(), color="red", s=8, label='Approximation')
            plt.title("Distribution") 
            plt.ylabel("Frequency")
            plt.yscale("log") 
            plt.ylim(1, max(distribution.values()))
            plt.xlabel("Triangle Participation")
            plt.xscale("log")
            plt.xlim(1, max(distribution.keys()))
            plt.legend()
            plt.show()
    
        times_apx_total[k] = times_apx_sampling[k] + times_apx_counting[k]
        
    if show_cmf:
        figure(num=None, figsize=(6, 4), dpi=150, facecolor='w', edgecolor='k')
        plt.title(f"CDF ({path})") 
        plt.plot(list(distribution_c.keys()), distribution_c.values(), color="blue", label='Exact distribution')
        for k, dist in approx_dist_c_k.items():
            if k <= 10 and k % 2 == 0:
                plt.plot(list(dist.keys()), dist.values(), label=f'k = {k}')
        plt.ylabel("Cumulative Probability")
        plt.xlabel("Triangle Participation")
        plt.xscale("log")
        plt.ylim(0,1)
        if out_path is not False:
            plt.savefig(out_path + 'cdf.png', bbox_inches='tight')
        plt.legend()
        plt.show()
    
    return ks_statistics, time_exact, times_apx_sampling, times_apx_counting, times_apx_total, times_cmf

In [ ]:
def run_experiment_x_times(path, k_arr, x):
    time_exact_i = []
    times_apx_sampling_i = {}
    times_apx_counting_i = {}
    times_apx_total_i = {}
    ks_statistics_i = {}
    
    for k in k_arr:
        times_apx_sampling_i[k] = []
        times_apx_counting_i[k] = []
        times_apx_total_i[k] = []
        ks_statistics_i[k] = []
        
    out_path = f'output/{path}/{round(time.time())}/'
    Path(out_path).mkdir(parents=True, exist_ok=True)
    
    for i in range(x):
        ks_statistics, time_exact, times_apx_sampling, times_apx_counting, times_apx_total, times_cmf = run_experiment(path, k_arr, out_path, True, False)
        time_exact_i.append(time_exact)
        for k in k_arr:
            times_apx_sampling_i[k].append(times_apx_sampling[k])
            times_apx_counting_i[k].append(times_apx_counting[k])
            times_apx_total_i[k].append(times_apx_total[k])
            ks_statistics_i[k].append(ks_statistics[k])
    
    # calculate medians
    time_exact = statistics.median(time_exact_i)
    times_apx_sampling = {}
    times_apx_counting = {}
    times_apx_total = {}
    ks_statistics = {}
    
    for k in k_arr:
        times_apx_sampling[k] = statistics.median(times_apx_sampling_i[k])
        times_apx_counting[k] = statistics.median(times_apx_counting_i[k])
        times_apx_total[k] = statistics.median(times_apx_total_i[k])
        ks_statistics[k] = statistics.median(ks_statistics_i[k])
    
    # display time performance
    figure(num=None, figsize=(6, 4), dpi=150, facecolor='w', edgecolor='k')
    plt.title(f'Running time ({path})')
    plt.hlines(y=time_exact, xmin=min(k_arr), xmax=max(k_arr), color='orange', label='Exact computation')
    plt.plot(list(times_apx_total.keys()), times_apx_total.values(), color="blue", label='Total approximation')
    plt.plot(list(times_apx_sampling.keys()), times_apx_sampling.values(), color="purple", label='Colorful sampling')
    plt.plot(list(times_apx_counting.keys()), times_apx_counting.values(), color="green", label='Triangle counting')
    plt.ylabel("time in seconds")
    plt.xlabel("k")
    plt.xticks(k_arr)
    plt.savefig(out_path + f'{x}-running_time.png', bbox_inches='tight')
    plt.legend()
    plt.show()
    
    # display precision(error)
    figure(num=None, figsize=(6, 4), dpi=150, facecolor='w', edgecolor='k')
    plt.title(f'Precision ({path})')
    plt.plot(list(ks_statistics.keys()), ks_statistics.values(), color="red")
    plt.ylabel("error(ks statistic)")
    plt.xlabel("k")
    plt.xticks(k_arr)
    plt.savefig(out_path + f'{x}-error.png', bbox_inches='tight')
    plt.show()
    
    return ks_statistics, time_exact, times_apx_sampling, times_apx_counting, times_apx_total

In [ ]:
%%time

# Files MOST NOT include any comments and should be stored in the subfolder "data"!
paths = [
    'web-NotreDame.txt',
    'web-Stanford.txt',
    'com-youtube.ungraph.txt',
    'ca-AstroPh.txt',
    'ca-HepPh.txt',
    'email-EuAll.txt',
    'email-Enron.txt',
    'amazon0302.txt',
    'roadNet-PA.txt',
    'soc-Slashdot0902.txt',
    'web-BerkStan.txt',
    'com-orkut.ungraph.txt'
    
    # todo: webbase
]

k_arr = [2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]

for path in paths:
    print(path)
    print(time.strftime("%d.%m.%Y %H:%M:%S"))
    run_experiment_x_times(path, k_arr, 10)

In [ ]:
# TODO: Remove duplicated code
def run_experiment_compare_colorings(path, k_arr, x):
    times_apx_sampling_a_i = {}
    times_apx_sampling_b_i = {}
    times_apx_sampling_c_i = {}
    ks_statistics_a_i = {}
    ks_statistics_b_i = {}
    ks_statistics_c_i = {}
    
    for k in k_arr:
        times_apx_sampling_a_i[k] = []
        times_apx_sampling_b_i[k] = []
        times_apx_sampling_c_i[k] = []
        ks_statistics_a_i[k] = []
        ks_statistics_b_i[k] = []
        ks_statistics_c_i[k] = []
        
    for i in range(x):
        global method
        
        method = 0
        ks_statistics, time_exact, times_apx_sampling, times_apx_counting, times_apx_total, times_cmf = run_experiment(path, k_arr, False, False, False)
        for k in k_arr:
            times_apx_sampling_a_i[k].append(times_apx_sampling[k])
            ks_statistics_a_i[k].append(ks_statistics[k])
          
        method = 1
        ks_statistics, time_exact, times_apx_sampling, times_apx_counting, times_apx_total, times_cmf = run_experiment(path, k_arr, False, False, False)
        for k in k_arr:
            times_apx_sampling_b_i[k].append(times_apx_sampling[k])
            ks_statistics_b_i[k].append(ks_statistics[k])
            
        method = 2
        ks_statistics, time_exact, times_apx_sampling, times_apx_counting, times_apx_total, times_cmf = run_experiment(path, k_arr, False, False, False)
        for k in k_arr:
            times_apx_sampling_c_i[k].append(times_apx_sampling[k])
            ks_statistics_c_i[k].append(ks_statistics[k])
    
    # calculate medians
    times_apx_sampling_a = {}
    times_apx_sampling_b = {}
    times_apx_sampling_c = {}
    ks_statistics_a = {}
    ks_statistics_b = {}
    ks_statistics_c = {}
    
    for k in k_arr:
        times_apx_sampling_a[k] = statistics.median(times_apx_sampling_a_i[k])
        times_apx_sampling_b[k] = statistics.median(times_apx_sampling_b_i[k])
        times_apx_sampling_c[k] = statistics.median(times_apx_sampling_c_i[k])
        ks_statistics_a[k] = statistics.median(ks_statistics_a_i[k])
        ks_statistics_b[k] = statistics.median(ks_statistics_b_i[k])
        ks_statistics_c[k] = statistics.median(ks_statistics_c_i[k])
    
    out_path = f'output/{path}/{round(time.time())}/'
    Path(out_path).mkdir(parents=True, exist_ok=True)
    
    # display time performance
    figure(num=None, figsize=(6, 4), dpi=80, facecolor='w', edgecolor='k')
    plt.title(f'Running time ({path})')
    plt.plot(list(times_apx_sampling_a.keys()), times_apx_sampling_a.values(), color="orange", label='Modulo Only')
    plt.plot(list(times_apx_sampling_b.keys()), times_apx_sampling_b.values(), color="purple", label='Permutation')
    plt.plot(list(times_apx_sampling_c.keys()), times_apx_sampling_c.values(), color="blue", label='Adler32')
    plt.ylabel("time in seconds")
    plt.xlabel("k")
    plt.ylim(0, max(times_apx_sampling_c.values()))
    plt.xticks(k_arr)
    plt.legend()
    plt.savefig(out_path + f'{x}-time-ab.png')
    plt.show()
    
    # display precision(error)
    figure(num=None, figsize=(6, 4), dpi=80, facecolor='w', edgecolor='k')
    plt.title(f'Error ({path})')
    plt.plot(list(ks_statistics_a.keys()), ks_statistics_a.values(), color="orange", label='Modulo Only')
    plt.plot(list(ks_statistics_b.keys()), ks_statistics_b.values(), color="purple", label='Permutation')
    plt.plot(list(ks_statistics_c.keys()), ks_statistics_c.values(), color="blue", label='Adler32')
    plt.ylabel("error(ks statistic)")
    plt.xlabel("k")
    plt.ylim(0, max(ks_statistics_c.values()))
    plt.xticks(k_arr)
    plt.legend()
    plt.savefig(out_path + f'{x}-error-ab.png')
    plt.show()

In [ ]:
%%time

paths = [
    'email-Enron.txt',
    'web-NotreDame.txt',
    'ca-AstroPh.txt',
]

k_arr = [2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]

for path in paths:
    print(path)
    run_experiment_compare_colorings(path, k_arr, 10)
    
# HACK: reset sampling method to not interfere with other experiments
method = 1

In [ ]:
# generate random graph
Gr = snap.GenRndGnm(snap.PUNGraph, 1000, 100000)

snap.SaveEdgeList(Gr, 'data/random.txt')
# HACK: remove comments from file manually.

print(f"Nodes: {Gr.GetNodes()}; Edges: {Gr.GetEdges()}; Triangles: {snap.GetTriads(Gr)}")

In [ ]:
# test random graph
k_arr = [2,3,4,5,6,7,8,9,10]
run_experiment_x_times('random.txt', k_arr, 10)